In [1]:
import pyodbc
import sqlalchemy
import urllib.request
import time
import os
def cnxn(dbName):
    cnxn_str = ("Driver=ODBC Driver 17 for SQL Server;"
                "Server=DESKTOP-4ABRK6A\SQLEXPRESS;"
                f"Database={dbName};"
                "UID=hemantcgupta;"
                "PWD=hemantcgupta")
    return pyodbc.connect(cnxn_str)

In [55]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
tickerName = 'HINDOILEXP'
query = f'''
SELECT 
    CONVERT(date, o.Datetime) AS Date, o.[Open], o.[High], o.[Low], o.[Close], o.[Volume],
    ((MAX(o.High) - MIN(o.Low)) / MAX(o.[Close])) AS BASR_Day,
    SUM(o.Volume) / COUNT(DISTINCT CONVERT(date, o.Datetime)) AS ATV_Day,
    ((MAX(o.High) - MIN(o.Low)) / MAX(o.[Close])) * LOG(SUM(o.Volume) / COUNT(DISTINCT CONVERT(date, o.Datetime))) AS LS_Day, 
	mif.nCandleBelowOpen, mif.pCandleAboveOpen, mif.nCandle, mif.pCandle, mif.Hits44MA,
	case when (sp.Entry2 <= sp.predTmEntry2 AND (sp.Exit2 >= sp.predTmExit2 OR sp.predTmEntry2 < sp.[Close])) then 'YES' else 'NO' end as entry_close,
	case when (sp.Entry2 <= sp.predTmEntry2 AND sp.Exit2 >= sp.predTmExit2) then 'YES' else 'NO' end as entry_exit,
	case when (sp.Entry2 <= sp.predTmEntry2 AND sp.predTmEntry2 >= sp.[Close]) then 'YES' else 'NO' end as entry_loss,
	sp.Entry2, sp.Exit2, sp.predTmEntry2, sp.predTmExit2, round(sp.Entry2-sp.predTmEntry2, 2) AS diff, round(sp.predTmEntry2-sp.[Close], 2) AS loss,
	round(sp.predTmEntry2-sp.[Low], 2) AS low_loss
FROM 
	{tickerName} AS o
LEFT JOIN (select * from mkanalyzer.dbo.mkIntervalFeature where tickerName = '{tickerName}') AS mif ON mif.Datetime = o.Datetime
LEFT JOIN (select * from mkanalyzer.dbo.simulationPrediction where tickerName = '{tickerName}') AS sp ON sp.predDatetime = o.Datetime
GROUP BY 
    CONVERT(date, o.Datetime), o.[Open], o.[High], o.[Low], o.[Close], o.[Volume], 
	mif.nCandleBelowOpen, mif.pCandleAboveOpen, mif.nCandle, mif.pCandle, mif.Hits44MA,
	sp.Entry2, sp.predTmEntry2, sp.Exit2, sp.predTmExit2, sp.[Close], sp.[Low]
ORDER BY
    Date DESC
'''
df = pd.read_sql(query, cnxn('mkdaymaster'))
df = df[['Date', 'BASR_Day', 'ATV_Day', 'LS_Day', 'nCandleBelowOpen', 'pCandleAboveOpen', 'nCandle', 'pCandle', 'Hits44MA', 'entry_exit', 'entry_close', 'entry_loss']]
df.head()

,Date,BASR_Day,ATV_Day,LS_Day,nCandleBelowOpen,pCandleAboveOpen,nCandle,pCandle,Hits44MA,entry_exit,entry_close,entry_loss
0,2024-07-19,0.068469,7034653,1.079504,75.0,0.0,46.0,29.0,13.0,NO,NO,YES
1,2024-07-18,0.081164,9062413,1.300218,45.0,30.0,33.0,42.0,13.0,YES,YES,NO
2,2024-07-16,0.046444,5425916,0.720190,1.0,74.0,38.0,37.0,4.0,YES,YES,NO
3,2024-07-15,0.046636,5412043,0.723057,134.0,28.0,92.0,70.0,54.0,NO,NO,NO
4,2024-07-12,0.055390,10463071,0.895296,1.0,74.0,38.0,37.0,13.0,YES,YES,NO


In [56]:
df['TmDate'] = df['Date']
df['TmPL'] = 0
df.loc[(df['entry_exit'] == 'YES') | (df['entry_close'] == 'YES'), 'TmPL'] = 1
df.head()

,Date,BASR_Day,ATV_Day,LS_Day,nCandleBelowOpen,pCandleAboveOpen,nCandle,pCandle,Hits44MA,entry_exit,entry_close,entry_loss,TmDate,TmPL
0,2024-07-19,0.068469,7034653,1.079504,75.0,0.0,46.0,29.0,13.0,NO,NO,YES,2024-07-19,0
1,2024-07-18,0.081164,9062413,1.300218,45.0,30.0,33.0,42.0,13.0,YES,YES,NO,2024-07-18,1
2,2024-07-16,0.046444,5425916,0.720190,1.0,74.0,38.0,37.0,4.0,YES,YES,NO,2024-07-16,1
3,2024-07-15,0.046636,5412043,0.723057,134.0,28.0,92.0,70.0,54.0,NO,NO,NO,2024-07-15,0
4,2024-07-12,0.055390,10463071,0.895296,1.0,74.0,38.0,37.0,13.0,YES,YES,NO,2024-07-12,1


In [57]:
df['TmDate'] = df['TmDate'].shift(1)
df['TmPL'] = df['TmPL'].shift(1)
df = df[~((df['entry_exit'] == 'NO') & (df['entry_close'] == 'NO') & (df['entry_loss'] == 'NO') & (df['TmPL'] == 0))].reset_index(drop=True)
df = df[['Date', 'TmDate', 'BASR_Day', 'ATV_Day', 'LS_Day', 'nCandleBelowOpen', 'pCandleAboveOpen', 'nCandle', 'pCandle', 'Hits44MA', 'TmPL']]
df['BASR_Day'] = df['BASR_Day'].round(4)
df['ATV_Day'] = df['ATV_Day'].round(4)
df['LS_Day'] = df['LS_Day'].round(4)
df = df.dropna().reset_index(drop=True)
df['TmPL'] = df['TmPL'].astype(int)

In [58]:
df

,Date,TmDate,BASR_Day,ATV_Day,LS_Day,nCandleBelowOpen,pCandleAboveOpen,nCandle,pCandle,Hits44MA,TmPL
0,2024-07-18,2024-07-19,0.0812,9062413,1.3002,45.0,30.0,33.0,42.0,13.0,0
1,2024-07-16,2024-07-18,0.0464,5425916,0.7202,1.0,74.0,38.0,37.0,4.0,1
2,2024-07-15,2024-07-16,0.0466,5412043,0.7231,134.0,28.0,92.0,70.0,54.0,1
3,2024-07-12,2024-07-15,0.0554,10463071,0.8953,1.0,74.0,38.0,37.0,13.0,0
4,2024-07-11,2024-07-12,0.0793,9213337,1.2711,13.0,62.0,34.0,41.0,12.0,1
...,...,...,...,...,...,...,...,...,...,...,...
91,2023-12-20,2023-12-21,0.1094,15664422,1.8123,17.0,58.0,36.0,39.0,6.0,0
92,2023-12-19,2023-12-20,0.0306,979889,0.4216,73.0,2.0,36.0,39.0,28.0,1
93,2023-12-18,2023-12-19,0.0396,1445027,0.5610,75.0,0.0,36.0,39.0,13.0,0
94,2023-12-15,2023-12-18,0.0578,5085968,0.8931,28.0,47.0,42.0,33.0,14.0,0


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Make predictions
y_pred = gnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 58.62%
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.70      0.70        20
           1       0.33      0.33      0.33         9

    accuracy                           0.59        29
   macro avg       0.52      0.52      0.52        29
weighted avg       0.59      0.59      0.59        29



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

# Convert features to integers for MultinomialNB
X = X.apply(pd.to_numeric, errors='coerce').astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features (MultinomialNB expects non-negative values, but we'll standardize for consistency)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ensure no negative values (optional but recommended for MultinomialNB)
X_train = X_train - X_train.min()
X_test = X_test - X_test.min()

# Initialize and train Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

# Make predictions
y_pred = mnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 46.15%
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.56      0.56        16
           1       0.30      0.30      0.30        10

    accuracy                           0.46        26
   macro avg       0.43      0.43      0.43        26
weighted avg       0.46      0.46      0.46        26



In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

# Convert features to binary (0 or 1) for BernoulliNB
X = (X > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train Bernoulli Naive Bayes
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

# Make predictions
y_pred = bnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 34.48%
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.15      0.24        20
           1       0.29      0.78      0.42         9

    accuracy                           0.34        29
   macro avg       0.45      0.46      0.33        29
weighted avg       0.50      0.34      0.30        29



In [93]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

# Convert features to float (if needed) for SVC
X = X.astype(float)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100, 150, 200],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']  # Only relevant for 'rbf' and 'poly' kernels
}

# Initialize GridSearchCV with SVC
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
print("Best Parameters:")
print(grid_search.best_params_)

# Make predictions with the best estimator
y_pred = grid_search.best_estimator_.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters:
{'C': 1, 'gamma': 'scale', 'kernel': 'poly'}
Accuracy: 62.07%
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.90      0.77        20
           1       0.00      0.00      0.00         9

    accuracy                           0.62        29
   macro avg       0.33      0.45      0.38        29
weighted avg       0.46      0.62      0.53        29



In [94]:
new_data = {
    'BASR_Day': [0.0464],
    'ATV_Day': [5425916],
    'LS_Day': [0.7201],
    'nCandleBelowOpen': [1.0],
    'pCandleAboveOpen': [74.0],
    'nCandle': [38.0],
    'pCandle': [37.0],
    'Hits44MA': [4]
}

new_df = pd.DataFrame(new_data)

# Prepare the new data (same preprocessing as the training data)
X_new = new_df.astype(float)
X_new = scaler.transform(X_new)  # Standardize the new data

# Make a prediction
prediction = grid_search.best_estimator_.predict(X_new)

print("Prediction for the new data:", prediction[0])

Prediction for the new data: 0


In [95]:
new_data = {
    'BASR_Day': [0.0391],
    'ATV_Day': [3049056],
    'LS_Day': [0.5839],
    'nCandleBelowOpen': [69.0],
    'pCandleAboveOpen': [6.0],
    'nCandle': [42.0],
    'pCandle': [33.0],
    'Hits44MA': [18]
}
new_df = pd.DataFrame(new_data)

# Prepare the new data (same preprocessing as the training data)
X_new = new_df.astype(float)
X_new = scaler.transform(X_new)  # Standardize the new data

# Make a prediction
prediction = grid_search.best_estimator_.predict(X_new)

print("Prediction for the new data:", prediction[0])

Prediction for the new data: 0


In [126]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

# Convert features to float (if needed)
X = X.astype(float)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [127]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)


In [128]:
# Hyperparameters
input_dim = X_train.shape[1]
output_dim = len(y.unique())
learning_rate = 0.001
num_epochs = 1000

# Initialize the model, loss function, and optimizer
model = SimpleNN(input_dim=input_dim, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/1000], Loss: 0.7299
Epoch [2/1000], Loss: 0.7099
Epoch [3/1000], Loss: 0.6966
Epoch [4/1000], Loss: 0.6766
Epoch [5/1000], Loss: 0.6643
Epoch [6/1000], Loss: 0.6439
Epoch [7/1000], Loss: 0.5783
Epoch [8/1000], Loss: 0.6586
Epoch [9/1000], Loss: 0.7576
Epoch [10/1000], Loss: 0.6987
Epoch [11/1000], Loss: 0.5225
Epoch [12/1000], Loss: 0.5606
Epoch [13/1000], Loss: 0.5184
Epoch [14/1000], Loss: 0.6958
Epoch [15/1000], Loss: 0.6654
Epoch [16/1000], Loss: 0.7588
Epoch [17/1000], Loss: 0.4155
Epoch [18/1000], Loss: 0.5725
Epoch [19/1000], Loss: 0.6399
Epoch [20/1000], Loss: 0.6691
Epoch [21/1000], Loss: 0.5555
Epoch [22/1000], Loss: 0.4390
Epoch [23/1000], Loss: 0.3801
Epoch [24/1000], Loss: 0.5991
Epoch [25/1000], Loss: 0.4308
Epoch [26/1000], Loss: 0.7011
Epoch [27/1000], Loss: 0.3497
Epoch [28/1000], Loss: 0.4490
Epoch [29/1000], Loss: 0.3624
Epoch [30/1000], Loss: 0.6608
Epoch [31/1000], Loss: 0.6647
Epoch [32/1000], Loss: 0.5850
Epoch [33/1000], Loss: 0.5246
Epoch [34/1000], Lo

Epoch [273/1000], Loss: 0.3235
Epoch [274/1000], Loss: 0.3249
Epoch [275/1000], Loss: 0.5932
Epoch [276/1000], Loss: 0.3337
Epoch [277/1000], Loss: 0.3137
Epoch [278/1000], Loss: 0.3224
Epoch [279/1000], Loss: 0.3463
Epoch [280/1000], Loss: 0.3680
Epoch [281/1000], Loss: 0.3430
Epoch [282/1000], Loss: 0.3139
Epoch [283/1000], Loss: 0.3149
Epoch [284/1000], Loss: 0.3136
Epoch [285/1000], Loss: 0.3138
Epoch [286/1000], Loss: 0.3516
Epoch [287/1000], Loss: 0.3367
Epoch [288/1000], Loss: 0.3141
Epoch [289/1000], Loss: 0.3158
Epoch [290/1000], Loss: 0.3186
Epoch [291/1000], Loss: 0.3133
Epoch [292/1000], Loss: 0.3192
Epoch [293/1000], Loss: 0.5356
Epoch [294/1000], Loss: 0.3133
Epoch [295/1000], Loss: 0.3148
Epoch [296/1000], Loss: 0.3133
Epoch [297/1000], Loss: 0.3133
Epoch [298/1000], Loss: 0.3259
Epoch [299/1000], Loss: 0.3139
Epoch [300/1000], Loss: 0.3134
Epoch [301/1000], Loss: 0.3180
Epoch [302/1000], Loss: 0.3259
Epoch [303/1000], Loss: 0.3186
Epoch [304/1000], Loss: 0.3287
Epoch [3

Epoch [542/1000], Loss: 0.3137
Epoch [543/1000], Loss: 0.3134
Epoch [544/1000], Loss: 0.3136
Epoch [545/1000], Loss: 0.3135
Epoch [546/1000], Loss: 0.3133
Epoch [547/1000], Loss: 0.3133
Epoch [548/1000], Loss: 0.3159
Epoch [549/1000], Loss: 0.3133
Epoch [550/1000], Loss: 0.3133
Epoch [551/1000], Loss: 0.3164
Epoch [552/1000], Loss: 0.3133
Epoch [553/1000], Loss: 0.3146
Epoch [554/1000], Loss: 0.3166
Epoch [555/1000], Loss: 0.3133
Epoch [556/1000], Loss: 0.3136
Epoch [557/1000], Loss: 0.3134
Epoch [558/1000], Loss: 0.3133
Epoch [559/1000], Loss: 0.3134
Epoch [560/1000], Loss: 0.3156
Epoch [561/1000], Loss: 0.3133
Epoch [562/1000], Loss: 0.3133
Epoch [563/1000], Loss: 0.3143
Epoch [564/1000], Loss: 0.3134
Epoch [565/1000], Loss: 0.3136
Epoch [566/1000], Loss: 0.3134
Epoch [567/1000], Loss: 0.3139
Epoch [568/1000], Loss: 0.3134
Epoch [569/1000], Loss: 0.3137
Epoch [570/1000], Loss: 0.3133
Epoch [571/1000], Loss: 0.3135
Epoch [572/1000], Loss: 0.3150
Epoch [573/1000], Loss: 0.3137
Epoch [5

Epoch [807/1000], Loss: 0.3133
Epoch [808/1000], Loss: 0.3133
Epoch [809/1000], Loss: 0.3133
Epoch [810/1000], Loss: 0.3133
Epoch [811/1000], Loss: 0.3133
Epoch [812/1000], Loss: 0.3133
Epoch [813/1000], Loss: 0.3133
Epoch [814/1000], Loss: 0.3133
Epoch [815/1000], Loss: 0.3133
Epoch [816/1000], Loss: 0.3133
Epoch [817/1000], Loss: 0.3133
Epoch [818/1000], Loss: 0.3133
Epoch [819/1000], Loss: 0.3133
Epoch [820/1000], Loss: 0.3133
Epoch [821/1000], Loss: 0.3133
Epoch [822/1000], Loss: 0.3133
Epoch [823/1000], Loss: 0.3133
Epoch [824/1000], Loss: 0.3133
Epoch [825/1000], Loss: 0.3133
Epoch [826/1000], Loss: 0.3133
Epoch [827/1000], Loss: 0.3133
Epoch [828/1000], Loss: 0.3133
Epoch [829/1000], Loss: 0.3133
Epoch [830/1000], Loss: 0.3133
Epoch [831/1000], Loss: 0.3133
Epoch [832/1000], Loss: 0.3133
Epoch [833/1000], Loss: 0.3134
Epoch [834/1000], Loss: 0.3133
Epoch [835/1000], Loss: 0.3133
Epoch [836/1000], Loss: 0.3133
Epoch [837/1000], Loss: 0.3133
Epoch [838/1000], Loss: 0.3133
Epoch [8

In [129]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 58.62%


In [134]:
# 0.0391114940877973	3049056	0.583948006034544	69	6	42	33	18	NO	NO	NO
# Prepare new data for prediction
new_data = {
    'BASR_Day': [0.0391],
    'ATV_Day': [3049056],
    'LS_Day': [0.5839],
    'nCandleBelowOpen': [69.0],
    'pCandleAboveOpen': [6.0],
    'nCandle': [42.0],
    'pCandle': [33.0],
    'Hits44MA': [18]
}

new_df = pd.DataFrame(new_data)
X_new = new_df.astype(float)
X_new = scaler.transform(X_new)
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

# Make prediction
model.eval()
with torch.no_grad():
    output = model(X_new_tensor)
    _, prediction = torch.max(output, 1)
    print("Prediction for the new data:", prediction.item())


tensor([1, 1, 0, 0, 1])


RuntimeError: a Tensor with 5 elements cannot be converted to Scalar

# Bagging

In [114]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

# Define features and target
X = df.drop(columns=['Date', 'TmDate', 'TmPL'])
y = df['TmPL']

# Convert features to float (if needed)
X = X.astype(float)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [115]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)


In [116]:
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, classification_report

class BaggingClassifier:
    def __init__(self, base_model, n_estimators=10):
        self.base_model = base_model
        self.n_estimators = n_estimators
        self.models = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(len(X), len(X), replace=True)
            X_resampled = X[indices]
            y_resampled = y[indices]
            
            model = self.base_model
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            # Training loop
            model.train()
            for epoch in range(5):  # Adjust number of epochs as needed
                for inputs, labels in DataLoader(TensorDataset(torch.tensor(X_resampled, dtype=torch.float32), torch.tensor(y_resampled, dtype=torch.long)), batch_size=4, shuffle=True):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            self.models.append(model)

    def predict(self, X):
        predictions = []
        for model in self.models:
            model.eval()
            with torch.no_grad():
                outputs = model(torch.tensor(X, dtype=torch.float32))
                _, predicted = torch.max(outputs, 1)
                predictions.append(predicted.numpy())
        
        # Aggregate predictions (majority voting)
        predictions = np.array(predictions)
        majority_vote = np.mean(predictions, axis=0)
        final_predictions = np.round(majority_vote).astype(int)
        return final_predictions


In [117]:
# Initialize and train the BaggingClassifier
base_model = SimpleNN(input_dim=X_train.shape[1], output_dim=len(y.unique()))
bagging_model = BaggingClassifier(base_model, n_estimators=1000)

# Ensure X_train and y_train_tensor are numpy arrays for compatibility
X_train_np = X_train
y_train_np = y_train_tensor.numpy()

bagging_model.fit(X_train_np, y_train_np)

# Make predictions
X_test_np = X_test
y_test_np = y_test_tensor.numpy()
y_pred = bagging_model.predict(X_test_np)

# Evaluate the model
accuracy = accuracy_score(y_test_np, y_pred)
print(f'Test Accuracy: {accuracy:.2f}%')
print("Classification Report:")
print(classification_report(y_test_np, y_pred))


Test Accuracy: 0.62%
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.55      0.67        20
           1       0.44      0.78      0.56         9

    accuracy                           0.62        29
   macro avg       0.64      0.66      0.61        29
weighted avg       0.72      0.62      0.63        29



In [118]:

# New data for prediction
new_data = {
    'BASR_Day': [0.0391],
    'ATV_Day': [3049056],
    'LS_Day': [0.5839],
    'nCandleBelowOpen': [69.0],
    'pCandleAboveOpen': [6.0],
    'nCandle': [42.0],
    'pCandle': [33.0],
    'Hits44MA': [18]
}

# Convert new data to DataFrame
new_df = pd.DataFrame(new_data)

# Standardize the new data
X_new = scaler.transform(new_df)  # Use the same scaler fitted on training data

# Convert to PyTorch tensor
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

# Make prediction
y_pred = bagging_model.predict(X_new)

print("Predictions for the new data:", y_pred)


Predictions for the new data: [1]


In [124]:
# New data for prediction
new_data = {
    'BASR_Day': [0.0568, 0.0695, 0.0700, 0.0267, 0.0489],
    'ATV_Day': [1233038, 1669825, 1777961, 1205204, 2194289],
    'LS_Day': [0.7962, 0.9969, 1.0074, 0.3744, 0.7152],
    'nCandleBelowOpen': [73, 73, 75, 74, 72],
    'pCandleAboveOpen': [2, 2, 0, 1, 3],
    'nCandle': [35, 38, 32, 37, 40],
    'pCandle': [40, 37, 43, 38, 35],
    'Hits44MA': [17, 7, 3, 15, 12]
}

# Convert new data to DataFrame
new_df = pd.DataFrame(new_data)
print(new_df)

# Standardize the new data
X_new = scaler.transform(new_df)  # Use the same scaler fitted on training data

# Convert to PyTorch tensor
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

# Make prediction
y_pred = bagging_model.predict(X_new)

print("Predictions for the new data:", y_pred)


   BASR_Day  ATV_Day  LS_Day  nCandleBelowOpen  pCandleAboveOpen  nCandle  \
0    0.0568  1233038  0.7962                73                 2       35   
1    0.0695  1669825  0.9969                73                 2       38   
2    0.0700  1777961  1.0074                75                 0       32   
3    0.0267  1205204  0.3744                74                 1       37   
4    0.0489  2194289  0.7152                72                 3       40   

   pCandle  Hits44MA  
0       40        17  
1       37         7  
2       43         3  
3       38        15  
4       35        12  
Predictions for the new data: [1 1 0 0 1]
